### ARIMA (AutoRegressive Integrated Moving Average) requires:

Stationary Data (Using differencing if necessary)

Hyperparameter Selection (p, d, q)

ACF & PACF Plots to select model parameters

Forecasting Future Prices using trained ARIMA

In [27]:
import pandas as pd
import numpy as np
import logging
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from typing import Tuple, Dict

warnings.filterwarnings("ignore")


class TimeSeriesAnalyzer:
    """A comprehensive time series analysis system for financial data."""

    def __init__(self, file_path: str, stock: str):
        """
        Initialize the TimeSeriesAnalyzer.

        Args:
            file_path (str): Path to the CSV file containing stock data.
            stock (str): Stock symbol to analyze.
        """
        self.file_path = file_path
        self.stock = stock
        self.data = None
        self.best_order = None
        self.arima_model = None
        self.sarima_model = None
        self.train_data = None
        self.test_data = None

        # Configure logging
        self._setup_logging()

    def _setup_logging(self) -> None:
        """Configure logging settings."""
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - %(levelname)s - %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S",
        )
        self.logger = logging.getLogger(__name__)

    def load_data(self, train_size: float = 0.8) -> pd.DataFrame:
        """Load and prepare time series data."""
        try:
            # Load data
            df = pd.read_csv(self.file_path, parse_dates=["Date"])
            df.set_index("Date", inplace=True)

            # Check if stock exists in dataset
            stock_col = f"{self.stock}_Close"
            if stock_col not in df.columns:
                raise ValueError(f"Column '{stock_col}' not found in the dataset!")

            # Set frequency and sort
            self.data = df[[stock_col]].asfreq("D")
            self.data.sort_index(inplace=True)

            # Handle missing values
            self.data = self.data.fillna(method="ffill").fillna(method="bfill")

            # Split data
            train_size = int(len(self.data) * train_size)
            self.train_data = self.data[:train_size]
            self.test_data = self.data[train_size:]

            self.logger.info(f"✅ Data Loaded for {self.stock}: {len(self.data)} records")
            return self.data

        except Exception as e:
            self.logger.error(f"❌ Error loading data: {str(e)}")
            raise

    def analyze_data(self) -> Dict:
        """Perform initial data analysis."""
        try:
            adf_result = adfuller(self.data[f"{self.stock}_Close"])
            analysis = {
                "total_samples": len(self.data),
                "date_range": f"{self.data.index.min()} to {self.data.index.max()}",
                "mean_price": self.data[f"{self.stock}_Close"].mean(),
                "std_price": self.data[f"{self.stock}_Close"].std(),
                "adf_statistic": adf_result[0],
                "adf_pvalue": adf_result[1],
                "is_stationary": adf_result[1] < 0.05,
            }
            return analysis

        except Exception as e:
            self.logger.error(f"❌ Error in data analysis: {str(e)}")
            raise

    def find_best_parameters(self) -> Tuple[int, int, int]:
        """Find optimal ARIMA parameters."""
        try:
            model = auto_arima(
                self.train_data[f"{self.stock}_Close"],
                seasonal=False,
                stepwise=True,
                suppress_warnings=True,
                error_action="ignore",
            )
            self.best_order = model.order
            self.logger.info(f"✅ Best ARIMA order: {self.best_order}")
            return self.best_order

        except Exception as e:
            self.logger.error(f"❌ Error finding best parameters: {str(e)}")
            raise

    def train_models(self, seasonal_order: Tuple[int, int, int, int] = (1, 1, 1, 12)) -> None:
        """Train ARIMA and SARIMA models."""
        try:
            if self.best_order is None:
                self.best_order = self.find_best_parameters()

            # Train ARIMA
            self.arima_model = ARIMA(
                self.train_data[f"{self.stock}_Close"], order=self.best_order
            ).fit()

            # Train SARIMA
            self.sarima_model = SARIMAX(
                self.train_data[f"{self.stock}_Close"],
                order=self.best_order,
                seasonal_order=seasonal_order,
            ).fit()

            self.logger.info("✅ Models trained successfully")

        except Exception as e:
            self.logger.error(f"❌ Error training models: {str(e)}")
            raise

    def evaluate_model(self, model, model_name: str) -> Dict:
        """Evaluate model performance."""
        try:
            # Generate predictions
            predictions = model.predict(
                start=self.test_data.index[0], end=self.test_data.index[-1]
            )

            # Ensure alignment
            actual_values = self.test_data[f"{self.stock}_Close"]
            predictions, actual_values = predictions.align(actual_values, join="inner")

            # Calculate metrics
            metrics = {
                "mae": mean_absolute_error(actual_values, predictions),
                "rmse": np.sqrt(mean_squared_error(actual_values, predictions)),
                "r2": r2_score(actual_values, predictions),
                "mape": np.mean(np.abs((actual_values - predictions) / actual_values)) * 100,
            }

            self.logger.info(f"✅ {model_name} Evaluation Done")
            return metrics

        except Exception as e:
            self.logger.error(f"❌ Error evaluating {model_name}: {str(e)}")
            raise


def main():
    """Main execution function."""
    try:
        # Load dataset
        file_path = "../data/processed/final_merged_stock_data.csv"
        df = pd.read_csv(file_path)

        # Identify stock symbols dynamically
        stock_symbols = [col.split("_")[0] for col in df.columns if "_Close" in col]

        print(f"✅ Detected stocks for ARIMA & SARIMA training: {stock_symbols}")

        for stock in stock_symbols:
            print(f"\n📊 Processing {stock}...")

            try:
                # Initialize analyzer
                analyzer = TimeSeriesAnalyzer(file_path, stock)

                # Load and analyze data
                analyzer.load_data()
                analysis = analyzer.analyze_data()

                print("\n📊 Initial Data Analysis:")
                for key, value in analysis.items():
                    print(f"{key}: {value}")

                # Train models
                analyzer.train_models()

                # Evaluate models
                arima_metrics = analyzer.evaluate_model(analyzer.arima_model, "ARIMA")
                sarima_metrics = analyzer.evaluate_model(analyzer.sarima_model, "SARIMA")

                # Print results
                print(f"\n🏆 {stock} Model Performance:")
                print(f"ARIMA RMSE: {arima_metrics['rmse']:.4f}")
                print(f"SARIMA RMSE: {sarima_metrics['rmse']:.4f}")

            except Exception as e:
                print(f"❌ Error processing {stock}: {e}")

        print("\n🏆 All stocks processed successfully!")

    except Exception as e:
        logging.error(f"❌ Error in main execution: {str(e)}")
        raise


if __name__ == "__main__":
    main()


2025-02-08 12:48:47,474 - INFO - ✅ Data Loaded for AAPL: 2920 records


✅ Detected stocks for ARIMA & SARIMA training: ['AAPL', 'AMZN', 'GOOGL', 'IBM', 'META', 'MSFT', 'NFLX', 'NVDA', 'ORCL', 'TSLA']

📊 Processing AAPL...

📊 Initial Data Analysis:
total_samples: 2920
date_range: 2015-01-02 00:00:00 to 2022-12-30 00:00:00
mean_price: 95.75009939639214
std_price: 32.60055192615524
adf_statistic: -1.0779745837131058
adf_pvalue: 0.7237749958529591
is_stationary: False


2025-02-08 12:48:49,282 - INFO - ✅ Best ARIMA order: (1, 1, 0)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.55227D+00    |proj g|=  2.84121D-01

At iterate    5    f=  1.42083D+00    |proj g|=  6.97815D-02

At iterate   10    f=  1.40658D+00    |proj g|=  9.82765D-03

At iterate   15    f=  1.40515D+00    |proj g|=  2.16596D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     18     23      1     0     0   9.903D-06   1.405D+00
  F =   1.4051457153584666     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


2025-02-08 12:48:52,212 - INFO - ✅ Models trained successfully
2025-02-08 12:48:52,230 - INFO - ✅ ARIMA Evaluation Done
2025-02-08 12:48:52,251 - INFO - ✅ SARIMA Evaluation Done
2025-02-08 12:48:52,300 - INFO - ✅ Data Loaded for AMZN: 2920 records


...


🏆 ORCL Model Performance:
ARIMA RMSE: 13.0921
SARIMA RMSE: 27.0810

📊 Processing TSLA...

📊 Initial Data Analysis:
total_samples: 2920
date_range: 2015-01-02 00:00:00 to 2022-12-30 00:00:00
mean_price: 95.21452031921945
std_price: 104.0368708192596
adf_statistic: -1.3122437470952293
adf_pvalue: 0.6235523349491868
is_stationary: False


2025-02-08 12:51:01,917 - INFO - ✅ Best ARIMA order: (0, 1, 0)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.46538D+00    |proj g|=  1.10992D-01

At iterate    5    f=  2.39476D+00    |proj g|=  3.78431D-02

At iterate   10    f=  2.38942D+00    |proj g|=  4.34490D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     14     17      1     0     0   5.662D-08   2.389D+00
  F =   2.3894127972278687     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


2025-02-08 12:51:03,365 - INFO - ✅ Models trained successfully
2025-02-08 12:51:03,382 - INFO - ✅ ARIMA Evaluation Done
2025-02-08 12:51:03,400 - INFO - ✅ SARIMA Evaluation Done



🏆 TSLA Model Performance:
ARIMA RMSE: 87.6417
SARIMA RMSE: 72.9781

🏆 All stocks processed successfully!


In [28]:
import joblib
import os
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Define paths
data_path = "../data/processed/final_merged_stock_data.csv"
save_dir = "../models/arima_sarima_models"

# Ensure save directory exists
os.makedirs(save_dir, exist_ok=True)

# Load dataset
df = pd.read_csv(data_path, parse_dates=["Date"])
df.set_index("Date", inplace=True)

# Identify stock symbols dynamically
stock_symbols = [col.split("_")[0] for col in df.columns if "_Close" in col]

print(f"✅ Detected stocks for saving models: {stock_symbols}")

# Loop through stocks and save models
for stock in stock_symbols:
    print(f"📌 Saving models for {stock}...")

    try:
        stock_col = f"{stock}_Close"
        if stock_col not in df.columns:
            print(f"🚨 Skipping {stock}: No closing price data found!")
            continue
        
        # Use past 80% data for training
        train_size = int(len(df) * 0.8)
        train_data = df[[stock_col]].iloc[:train_size]

        # Load trained ARIMA model
        arima_model = ARIMA(train_data, order=(1,1,0)).fit()
        joblib.dump(arima_model, os.path.join(save_dir, f"{stock}_ARIMA.pkl"))

        # Load trained SARIMA model
        sarima_model = SARIMAX(train_data, order=(1,1,0), seasonal_order=(1,1,1,12)).fit()
        joblib.dump(sarima_model, os.path.join(save_dir, f"{stock}_SARIMA.pkl"))

        print(f"✅ Models saved successfully for {stock}!")

    except Exception as e:
        print(f"❌ Error saving models for {stock}: {e}")

print("\n🏆 All models saved successfully!")


✅ Detected stocks for saving models: ['AAPL', 'AMZN', 'GOOGL', 'IBM', 'META', 'MSFT', 'NFLX', 'NVDA', 'ORCL', 'TSLA']
📌 Saving models for AAPL...
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.55227D+00    |proj g|=  2.84121D-01


 This problem is unconstrained.



At iterate    5    f=  1.42083D+00    |proj g|=  6.97815D-02

At iterate   10    f=  1.40658D+00    |proj g|=  9.82765D-03

At iterate   15    f=  1.40515D+00    |proj g|=  2.16596D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     18     23      1     0     0   9.903D-06   1.405D+00
  F =   1.4051457153584666     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
✅ Models saved successfully for AAPL!
📌 Saving models for AMZN...
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At 

 This problem is unconstrained.



At iterate    5    f=  1.59084D+00    |proj g|=  5.19844D-02

At iterate   10    f=  1.57303D+00    |proj g|=  3.76821D-03

At iterate   15    f=  1.57225D+00    |proj g|=  1.97940D-03

At iterate   20    f=  1.57222D+00    |proj g|=  2.14335D-04

At iterate   25    f=  1.57222D+00    |proj g|=  6.65779D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     26     33      1     0     0   5.489D-05   1.572D+00
  F =   1.5722169812393598     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
✅ Models saved successfully for AMZN!
📌 Saving models for GOOGL...
RUNNING THE L-BFGS-B CODE

           * * *

M

...


At iterate    5    f= -5.45833D-01    |proj g|=  7.14030D-01

At iterate   10    f= -5.89819D-01    |proj g|=  3.84776D-01

At iterate   15    f= -5.99531D-01    |proj g|=  1.02291D-01

At iterate   20    f= -6.02031D-01    |proj g|=  3.81742D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     24     35      1     0     0   2.301D-04  -6.020D-01
  F = -0.60203246722752379     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
✅ Models saved successfully for NVDA!
📌 Saving models for ORCL...


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.04647D-01    |proj g|=  6.39120D-01

At iterate    5    f=  7.07423D-01    |proj g|=  8.31945D-02

At iterate   10    f=  6.46076D-01    |proj g|=  1.09712D-01

At iterate   15    f=  6.39376D-01    |proj g|=  7.75580D-03

At iterate   20    f=  6.39283D-01    |proj g|=  3.26064D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     22     26      1     0     0   7.369D-06   6.393D-01
  F =  0.63928281638491746     

CONVERG

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.53241D+00    |proj g|=  1.12907D-01

At iterate    5    f=  2.40258D+00    |proj g|=  2.81815D-02

At iterate   10    f=  2.38993D+00    |proj g|=  8.16244D-03

At iterate   15    f=  2.38941D+00    |proj g|=  3.02455D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     16     17      1     0     0   7.454D-06   2.389D+00
  F =   2.3894093951718851     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
✅ Models

### ARIMA(2,1,2) model has been trained on AAPL_Close prices 

### Gradient Boosting Requires:

Feature Engineering (Lagged Prices, Rolling Mean, Volatility)

Train-Test Split for Supervised Learning

Hyperparameter Tuning using Grid Search

Evaluation Metrics (RMSE, MAE)

In [ ]:
import pandas as pd
import os
import numpy as np
import joblib
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import warnings

warnings.filterwarnings("ignore")

# ✅ **Prepare Data for ML**
def prepare_ml_data(df, stock):
    """Prepare features and target for Gradient Boosting."""
    col_name = f"{stock}_Close"

    # ✅ **Target Variable (Next Day's Closing Price)**
    df[f"{stock}_Target"] = df[col_name].shift(-1)

    # ✅ **Feature Selection (Drop Over-Dominant Features)**
    drop_cols = [f"{stock}_Adj Close"]  # Reduce importance of adjusted close price
    feature_cols = [col for col in df.columns if stock in col and col != f"{stock}_Target" and col not in drop_cols]

    df = df.dropna()  # Drop last row (NaN target)
    X = df[feature_cols]
    y = df[f"{stock}_Target"]

    return X, y

# ✅ **Hyperparameter Tuning & Training Functions**
def train_model(model_type, X_train, y_train):
    """Train & optimize models using RandomizedSearchCV."""

    param_dist = {
        "n_estimators": [100, 300, 500],
        "learning_rate": [0.01, 0.05, 0.1],
        "max_depth": [3, 5, 7],
        "subsample": [0.7, 0.9, 1.0],
    }

    if model_type == "GradientBoosting":
        model = GradientBoostingRegressor(random_state=42)
        param_dist.pop("colsample_bytree", None)  # REMOVE invalid parameter
    
    elif model_type == "XGBoost":
        model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
        param_dist["colsample_bytree"] = [0.6, 0.8, 1.0]  # Only for XGBoost
    
    elif model_type == "LightGBM":
        model = lgb.LGBMRegressor(random_state=42)
        param_dist["colsample_bytree"] = [0.6, 0.8, 1.0]  # Only for LightGBM

    random_search = RandomizedSearchCV(
        model, param_distributions=param_dist, n_iter=20, cv=3, scoring="neg_mean_squared_error", n_jobs=-1
    )
    random_search.fit(X_train, y_train)
    
    return random_search.best_estimator_

# ✅ **Evaluate Model Performance**
def evaluate_model(model, X_test, y_test, model_name, stock):
    """Evaluate model performance and return metrics."""
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"\n✅ {model_name} Model Evaluation for {stock}:")
    print(f"📉 RMSE: {rmse:.4f}")
    print(f"📉 MAE: {mae:.4f}")
    print(f"📈 R² Score: {r2:.4f}")

    return {"stock": stock, "model": model_name, "rmse": rmse, "mae": mae, "r2": r2}

# ✅ **Main Training Pipeline**
if __name__ == "__main__":
    feature_dir = "../data/feature_engineering data"
    model_save_dir = "../models/boosting_models"

    os.makedirs(model_save_dir, exist_ok=True)

    stock_files = [f for f in os.listdir(feature_dir) if f.endswith("_boosting_features.csv")]
    stock_symbols = [f.split("_")[0] for f in stock_files]

    print(f"✅ Detected feature datasets for stocks: {stock_symbols}")

    results = []

    for stock, file in zip(stock_symbols, stock_files):
        print(f"\n📊 Training models for {stock}...")

        file_path = os.path.join(feature_dir, file)
        df = pd.read_csv(file_path)

        # ✅ **Prepare Data**
        X, y = prepare_ml_data(df, stock)

        # ✅ **Train-Test Split**
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # ✅ **Train Models**
        gb_model = train_model("GradientBoosting", X_train, y_train)
        xgb_model = train_model("XGBoost", X_train, y_train)
        lgb_model = train_model("LightGBM", X_train, y_train)

        # ✅ **Evaluate Models**
        gb_results = evaluate_model(gb_model, X_test, y_test, "Gradient Boosting", stock)
        xgb_results = evaluate_model(xgb_model, X_test, y_test, "XGBoost", stock)
        lgb_results = evaluate_model(lgb_model, X_test, y_test, "LightGBM", stock)

        results.extend([gb_results, xgb_results, lgb_results])

        # ✅ **Save Models**
        joblib.dump(gb_model, os.path.join(model_save_dir, f"{stock}_GradientBoosting.pkl"))
        joblib.dump(xgb_model, os.path.join(model_save_dir, f"{stock}_XGBoost.pkl"))
        joblib.dump(lgb_model, os.path.join(model_save_dir, f"{stock}_LightGBM.pkl"))

        print(f"✅ Models saved for {stock} in {model_save_dir}")

    # ✅ **Convert Results to DataFrame**
    results_df = pd.DataFrame(results)
    results_df.to_csv("../data/model_performance_comparison.csv", index=False)

    # ✅ **Find Best Model Per Stock**
    best_models = results_df.groupby("stock").apply(lambda x: x.nsmallest(1, "rmse")).reset_index(drop=True)

    print("\n🏆 **Best Models for Each Stock**")
    print(best_models)

    # ✅ **Save Best Model Results**
    best_models.to_csv("../data/best_models.csv", index=False)
    print("\n🏆 Best models saved to `best_models.csv`")


✅ Detected feature datasets for stocks: ['NVDA', 'MSFT', 'TSLA', 'AAPL', 'GOOGL', 'AMZN', 'ORCL', 'IBM', 'META', 'NFLX']

📊 Training models for NVDA...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001215 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5585
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001115 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5585
[LightGBM] [Info] Number of data points in the train set: 582, number of used features: 29
[LightGBM] [Info] Number of data points in the train set: 582, number of used features: 29
[LightGBM] [Info] Start training from score 16.235139
[LightGBM] [Info] Start training from score 15.781509
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000916 seconds

### we will select the best gradient model for prediction Lightbgm 



In [26]:

import joblib

# Save LightGBM model
joblib.dump(lgb_model, "lightgbm_model.pkl")

print("✅ LightGBM model saved as 'lightgbm_model.pkl'")


✅ LightGBM model saved as 'lightgbm_model.pkl'


In [29]:
import joblib

# Save LightGBM model
joblib.dump(xgb_model, "xbg_model.pkl")

print("✅ LightGBM model saved as 'lightgbm_model.pkl'")

✅ LightGBM model saved as 'lightgbm_model.pkl'
